In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import ResNet50 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import cv2
import shutil
import time
from sklearn.metrics import classification_report

In [29]:
# Path dataset
train_csv = "./content/Training_set.csv"
train_folder = "./content/train/"

test_csv = "./content/Testing_set.csv"
test_folder = "./content/test/"

In [30]:
## Hyperparameter
image_size = (150, 150)
batch_size = 32
epochs = 100
learning_rate = 0.0001

In [31]:
checkpoint_path ="./best_model.h5"

In [32]:
#loading data
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

In [33]:
#data preprocessing
class_name = list(set(train_df['label']))

# Data preprocessing using list comprehensions
features = [cv2.resize(cv2.imread(os.path.join(train_folder, image)), image_size) / 255.0
            for image in tqdm(os.listdir(train_folder), desc="Preprocess Image")]

labels = [class_name.index(train_df.loc[train_df['filename'] == image, 'label'].values[0])
          for image in tqdm(os.listdir(train_folder), desc="Extract Labels")]

# Convert lists to NumPy arrays
features = np.array(features)
labels = np.array(labels)    

Extract Labels: 100%|██████████| 6499/6499 [00:12<00:00, 514.03it/s]


In [35]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, shuffle=True, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, shuffle=True, random_state=42)

In [36]:
base_model = ResNet50(
    weights='imagenet', 
    include_top=False, 
    input_shape=(150, 150, 3),
)

num_layers_to_train = int(np.ceil(0.2 * len(base_model.layers)))

for layer in base_model.layers[:num_layers_to_train] :
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
predictions = Dense(75, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 156, 156, 3)          0         ['input_5[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 75, 75, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 75, 75, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [ ]:
model.compile(optimizer=Adam(learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True)

start_time = time.time()

history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    validation_data = (X_valid,y_valid),
    callbacks=[model_checkpoint, early_stopping],
    batch_size = batch_size,
)

end_time = time.time()

Epoch 1/100
143/143 [==============================] - ETA: 0s - loss: 7.3307 - accuracy: 0.3548

C:\Users\ghamm\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


143/143 [==============================] - 628s 4s/step - loss: 7.3307 - accuracy: 0.3548 - val_loss: 8.3516 - val_accuracy: 0.0308
Epoch 2/100
143/143 [==============================] - 540s 4s/step - loss: 4.3101 - accuracy: 0.7874 - val_loss: 6.5705 - val_accuracy: 0.2103
Epoch 3/100
143/143 [==============================] - 517s 4s/step - loss: 3.0669 - accuracy: 0.9319 - val_loss: 4.1894 - val_accuracy: 0.6103
Epoch 4/100
 70/143 [=============>................] - ETA: 3:49 - loss: 2.5153 - accuracy: 0.9763